In [12]:
import numpy as np

FileName = "ims1998280_00UTC_24km_v1.1"

with open(FileName + ".asc", "r") as f:
    # Skip 1365 bytes within the header.
    # Take all lines after the header, strip the end-line character and join all in one string.
    f.seek(1365)
    Lines = " ".join(line.strip("\n") for line in f)

# Remove all white space from Lines with Lines.split.
# If characters (s) in string are digits, convert to integers, then convert to a NumPy Array.
# Lastly, reshape to 1024 x 1024 24 km resolution.
DataArray = np.reshape(
    np.asarray([int(s) for s in Lines.split() if s.isdigit()]), [1024, 1024]
)

# From the documentation (Table 3):
# For unpacked data, integer value of 164 is sea ice, while 165 is snow-covered land.
# Convert 164 to 3 (sea ice) and 165 to 4 (snow covered land) to align with packed data.
DataArray = np.where(DataArray == 164, 3, DataArray)
DataArray = np.where(DataArray == 165, 4, DataArray)

# Create the output header to be saved to the new file.
HeaderText = "NCOLS 1024\nNROWS 1024\nXLLCORNER -12126597.0\nYLLCORNER -12126840.0\nCELLSIZE 23684.997"
np.savetxt(
    FileName + "_2packed.asc",
    DataArray,
    header=HeaderText,
    delimiter=" ",
    fmt="%s",
    comments="",
)

In [15]:
type(DataArray), DataArray

(numpy.ndarray,
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

Initial Data Exploration. IMS Data is stored in .asc files, which contain: (1) header with information about the data collected, and (2) an array of float, which represent different qualities of data detailed:

- 0 (outside Northern Hemisphere).
- 1 (open water)
- 2 (land without snow)
- 3 (sea or lake ice)
- 4 (snow covered land)

In [16]:
DataArray.shape

(1024, 1024)

The DataArray containes significantly more 0 values than other numerical values. In order to determine if there are instances of other numerrical values I ran the following lines of code below:

In [17]:
# Below are instances of sea and land
unique, counts = np.unique(DataArray, return_counts=True)
dict(zip(unique, counts))

{0: 1015028, 3: 10163, 4: 23385}